## Constructing fasta files from Patric's data

In [ ]:
import os
import pandas as pd

# Specify the directory containing the TSV files
tsv_directory = "E:\\User\\bruna.fistarol\\HMM\\Salmonella\\Strains"

# Iterate over each file in the specified directory
for filename in os.listdir(tsv_directory):
    if filename.endswith(".tsv"):
        file_path = os.path.join(tsv_directory, filename)
        
        # Read the TSV file
        df = pd.read_csv(file_path, sep='\t')
        
        # Filter the dataframe for forward sequences only
        fwd_df = df[df['feature.feature_id'].str.endswith('.fwd')]
        
        # Construct the FASTA format string
        fasta_str = ""
        for index, row in fwd_df.iterrows():
            fasta_str += ">{}\n{}\n".format(row['feature.plfam_id'], row['feature.na_sequence'])
        
        # Write the FASTA format string to a new file
        fasta_filename = filename.replace(".tsv", ".fasta")
        fasta_filepath = os.path.join(tsv_directory, fasta_filename)
        with open(fasta_filepath, "w") as fasta_file:
            fasta_file.write(fasta_str)
        
        # Delete the TSV file
        os.remove(file_path)


## Identifying potentialy incomplete genomes by counting

In [ ]:
fasta_directory = "E:\\User\\bruna.fistarol\\HMM\\Salmonella\\Strains"

gene_counts = {}

for filename in os.listdir(fasta_directory):
    if filename.endswith(".fasta"):
        file_path = os.path.join(fasta_directory, filename)
        gene_count = 0
        
        with open(file_path, "r") as fasta_file:
            for line in fasta_file:
                if line.startswith(">"):
                    gene_count += 1
                    
        gene_counts[filename] = gene_count

# Determine threshold for low gene count
average_gene_count = sum(gene_counts.values()) / len(gene_counts)
threshold = average_gene_count * 0.8  # Adjust as necessary

# Identify genomes with low gene count
incomplete_genomes = [genome for genome, count in gene_counts.items() if count < threshold]
print("Potential Incomplete Genomes (Low Gene Count):", incomplete_genomes)

## Core genes

In [ ]:
from collections import Counter

# Specify the directory containing the FASTA files
fasta_directory = "E:\\User\\bruna.fistarol\\HMM\\Salmonella\\Strains"

# Initialize a Counter to store the occurrence of each protein family
protein_family_counter = Counter()
total_files = 0

# Initialize a dictionary to store the sequences of core protein families
core_protein_families = {}

# Step 1: Parse each FASTA file and count protein family occurrences
for filename in os.listdir(fasta_directory):
    if filename.endswith(".fasta"):
        total_files += 1
        file_path = os.path.join(fasta_directory, filename)
        
        with open(file_path, "r") as fasta_file:
            for line in fasta_file:
                if line.startswith(">"):
                    protein_family = line.strip().split(">")[1]
                    protein_family_counter[protein_family] += 1

# Step 2: Identify core protein families
threshold = total_files * 0.99  # 99% of all files
core_protein_families = {k: v for k, v in protein_family_counter.items() if v > threshold}

# Display core protein families
print("Core Protein Families (Present in more than 99% of files):")
for family in core_protein_families:
    print(family)

# Optional: Extract sequences of core protein families and save them
for protein_family in core_protein_families:
    # Creating a new file for each core protein family
    output_filepath = os.path.join(fasta_directory, f"{protein_family}.fasta")
    
    with open(output_filepath, "w") as output_file:
        # Parsing each FASTA file again to extract sequences
        for filename in os.listdir(fasta_directory):
            if filename.endswith(".fasta"):
                file_path = os.path.join(fasta_directory, filename)
                
                with open(file_path, "r") as fasta_file:
                    write_sequence = False
                    for line in fasta_file:
                        if line.startswith(">"):
                            current_protein_family = line.strip().split(">")[1]
                            if current_protein_family == protein_family:
                                write_sequence = True
                                output_file.write(line)
                            else:
                                write_sequence = False
                        elif write_sequence:
                            output_file.write(line)